In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D # type: ignore
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
import json
from tensorflow.keras.models import load_model # type: ignore


df = pd.read_csv('cocomo81_preprocessed.csv')
df.describe()

features = ['rely', 'data', 'stor', 'turn', 'aexp', 'pcap', 'lexp', 'modp', 'loc']
target = 'actual'
X = df[features].values
y = df[target].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'X_train size: {X_train.shape}')
print(f'X_test size: {X_val.shape}')

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val_cnn = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

# Build model
model = Sequential([
    Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=32, kernel_size=2, activation='relu'),
    Flatten(),
    Dense(20, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=30,
    restore_best_weights=True
)

model.fit(
    X_train_cnn, y_train,
    validation_data=(X_val_cnn, y_val),
    epochs=300,
    batch_size=8,
    callbacks=[early_stopping]
)

# Define bound for each parameter
param_bounds = {         
    'filters1': (8, 32),
    'filters2': (16, 64),
    'kernels1': (2, 5),
    'kernels2': (2, 4),  
    'dense_size': (1, 3),
    'neurons': (4, 64),
    'dropout_rate': (0.1, 0.5),
    'learning_rate': (0.0001, 0.01),
}
optimizer_list = ['adam', 'rmsprop']



# Define CNN model building function
def build_cnn_model(params, optimizer_name):
    model = tf.keras.Sequential()
    model.add(Conv1D(
        filters=int(round(params['filters1'])), 
        kernel_size=int(round(params['kernels1'])), 
        activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(
        filters=int(round(params['filters2'])), 
        kernel_size=int(round(params['kernels2'])), 
        activation='relu'))
    model.add(Flatten())
    
    for _ in range(int(round(params['dense_size']))):
        model.add(tf.keras.layers.Dense(int(round(params['neurons'])), activation='relu'))

    model.add(tf.keras.layers.Dropout(params['dropout_rate']))
    model.add(tf.keras.layers.Dense(1))

    optimizer = {
        'adam': tf.keras.optimizers.Adam,
        'rmsprop': tf.keras.optimizers.RMSprop
    }[optimizer_name](learning_rate=params['learning_rate'])

    model.compile(optimizer=optimizer,
                  loss='mse',
                  metrics=['mae'])
    return model




# Define model evaluation function
def evaluate_model(params, optimizer_idx, k=5):
    optimizer = optimizer_list[int(round(optimizer_idx))]

    X_cnn = X.reshape(X.shape[0], X.shape[1], 1)
    val_scores = []
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    for train_index, val_index in kf.split(X_cnn):
        X_train_v, X_val_v = X[train_index], X[val_index]
        y_train_v, y_val_v = y[train_index], y[val_index]
        
        scaler = StandardScaler()
        X_train_v = scaler.fit_transform(X_train_v)
        X_val_v = scaler.transform(X_val_v)
        
        model = build_cnn_model(params, optimizer)
        
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=30,
            restore_best_weights=True
        )

        history = model.fit(
            X_train_v, y_train_v,
            validation_data=(X_val_v, y_val_v),
            epochs=300,
            batch_size=8,
            verbose=0,
            callbacks=[early_stopping]
        )
        val_mae = min(history.history['val_mae'])
        val_scores.append(val_mae)

    return np.mean(val_scores)





# Define particle class
class Particle:
    def __init__(self):
        self.position = {
            'filters1': np.random.randint(*param_bounds['filters1']),
            'filters2': np.random.randint(*param_bounds['filters2']),
            'kernels1': np.random.randint(*param_bounds['kernels1']),
            'kernels2': np.random.randint(*param_bounds['kernels2']),
            'dense_size': np.random.randint(*param_bounds['dense_size']),
            'neurons': np.random.randint(*param_bounds['neurons']),
            'dropout_rate': np.random.uniform(*param_bounds['dropout_rate']),
            'learning_rate': np.random.uniform(*param_bounds['learning_rate']),
            'optimizer_idx': random.randint(0, len(optimizer_list) - 1),
        }
        self.velocity = {key: 0 for key in self.position}
        self.best_position = dict(self.position)
        self.best_score = float('inf')




# Define pso function
# https://arxiv.org/pdf/2101.11944
def pso(n_particles=5, n_iterations=10, w=0.5, c1=1.9, c2=1.9):
    swarm = [Particle() for _ in range(n_particles)]
    global_best = {'score': float('inf'), 'position': None}

    for iteration in range(n_iterations):
        print(f"Iteration {iteration+1}/{n_iterations}", end=" ")
        for particle in swarm:
            score = evaluate_model(particle.position, particle.position['optimizer_idx'])

            if score < particle.best_score:
                particle.best_score = score
                particle.best_position = dict(particle.position)

            if score < global_best['score']:
                global_best['score'] = score
                global_best['position'] = dict(particle.position)

        for particle in swarm:
            for key in particle.position:
                r1, r2 = np.random.rand(), np.random.rand()
                cognitive = c1 * r1 * (particle.best_position[key] - particle.position[key])
                social = c2 * r2 * (global_best['position'][key] - particle.position[key])
                particle.velocity[key] = w * particle.velocity[key] + cognitive + social
                particle.position[key] += particle.velocity[key]

                if key in param_bounds:
                    low, high = param_bounds[key]
                    particle.position[key] = np.clip(particle.position[key], low, high)

                elif key.endswith('_idx'):
                    particle.position[key] = int(np.clip(particle.position[key], 0, len(optimizer_list) - 1))
        print(f"MAE: {global_best['score']}")
    return global_best

best_result = pso(n_particles=10, n_iterations=10)
print("Best MAE:", best_result['score'])
print("Best Hyperparameters:")
for key, value in best_result['position'].items():
    if key.endswith('_idx'):
        print("optimizer:", optimizer_list[int(round(value))])
    else:
        print(f"{key}: {value}")



params = best_result['position']
optimizer = optimizer_list[int(best_result['position']['optimizer_idx'])]

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=30,
    restore_best_weights=True
)

model = build_cnn_model(params, optimizer)
history = model.fit(
    X_train_cnn, y_train,
    validation_data=(X_val_cnn, y_val),
    epochs=300,
    batch_size=8,
    callbacks=[early_stopping]
)


y_pred_val_log = model.predict(X_val_cnn)

# flatten to ensure it's a 1D array
y_pred_log = y_pred_val_log.flatten()

# avoid divide by 0
y_val_nonzero_log = y_val[y_val != 0]
y_pred_nonzero_log = y_pred_log[y_val != 0]

mse = mean_squared_error(y_val, y_pred_log)
mae = mean_absolute_error(y_val, y_pred_log)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred_log)

mape = np.mean(np.abs((y_val_nonzero_log - y_pred_nonzero_log) / y_val_nonzero_log))

mmre = np.mean(np.abs((y_val_nonzero_log - y_pred_nonzero_log) / y_val_nonzero_log))

mdmre = np.median(np.abs((y_val_nonzero_log - y_pred_nonzero_log) / y_val_nonzero_log))

pred_25 = np.mean(np.abs((y_val_nonzero_log - y_pred_nonzero_log) / y_val_nonzero_log) <= 0.25)

print(f"MSE         : {mse:.4f}")
print(f"MAE         : {mae:.4f}")
print(f"RMSE        : {rmse:.4f}")
print(f"R²          : {r2:.4f}")
print(f"MAPE        : {mape * 100:.2f}%")
print(f"MMRE        : {mmre:.4f}")
print(f"MdMRE       : {mdmre:.4f}")
print(f"PRED(25)    : {pred_25 * 100:.2f}%")


print()
n_bootstraps = 500
bootstrap_metrics = {'mse': [], 'mae': [], 'r2': [], 'mape': [], 'mmre': [], 'mdmre': [], 'pred25': []}
for _ in range(n_bootstraps):
    indices = np.random.choice(len(y_val), len(y_val), replace=True)
    y_val_boot = y_val[indices]
    y_pred_boot = y_pred_log[indices]
    bootstrap_metrics['mse'].append(mean_squared_error(y_val_boot, y_pred_boot))
    bootstrap_metrics['mae'].append(mean_absolute_error(y_val_boot, y_pred_boot))
    bootstrap_metrics['r2'].append(r2_score(y_val_boot, y_pred_boot))
    y_val_boot_nonzero = y_val_boot[y_val_boot != 0]
    y_pred_boot_nonzero = y_pred_boot[y_pred_boot != 0]
    bootstrap_metrics['mape'].append(np.mean(np.abs((y_val_boot_nonzero - y_pred_boot_nonzero) / y_val_boot_nonzero)))
    bootstrap_metrics['mmre'].append(np.mean(np.abs((y_val_boot_nonzero - y_pred_boot_nonzero) / y_val_boot_nonzero)))
    bootstrap_metrics['mdmre'].append(np.median(np.abs((y_val_boot_nonzero - y_pred_boot_nonzero) / y_val_boot_nonzero)))
    bootstrap_metrics['pred25'].append(np.mean(np.abs((y_val_boot_nonzero - y_pred_boot_nonzero) / y_val_boot_nonzero) <= 0.25))
    
print(f"MSE         : {np.std(bootstrap_metrics['mse']):.4f}")
print(f"MAE         : {np.std(bootstrap_metrics['mae']):.4f}")
print(f"RMSE        : {np.std(np.sqrt(bootstrap_metrics['mse'])):.4f}")
print(f"R²          : {np.std(bootstrap_metrics['r2']):.4f}")
print(f"MAPE        : {np.std(bootstrap_metrics['mape']) * 100:.2f}%")
print(f"MMRE        : {np.std(bootstrap_metrics['mmre']):.4f}")
print(f"MdMRE       : {np.std(bootstrap_metrics['mdmre']):.4f}")
print(f"PRED(25)    : {np.std(bootstrap_metrics['pred25'])* 100:.2f}%")

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

fig, axs = plt.subplots(2, 2, figsize=(14, 10))

# Loss
plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Huber Loss')
plt.legend()

# Predicted vs Actual
plt.subplot(2, 2, 2)
plt.scatter(y_val, y_pred_log, alpha=0.5)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--')
plt.title('Predicted vs Actual Effort (Scaled)')
plt.xlabel('Actual Effort (Scaled)')
plt.ylabel('Predicted Effort (Scaled)')

# Error Distribution
errors = y_val - y_pred_log
plt.subplot(2, 2, 3)
sns.histplot(errors, kde=True)
plt.title('Error Distribution')
plt.xlabel('Prediction Error (Scaled)')
plt.ylabel('Frequency')

# Bootstrap RMSE
plt.subplot(2, 2, 4)
sns.boxplot(y=np.sqrt(bootstrap_metrics['mse']))
plt.title('Bootstrap RMSE Distribution (Scaled)')
plt.ylabel('RMSE (Scaled)')

plt.tight_layout()
plt.show()

X_train size: (50, 9)
X_test size: (13, 9)


c:\Users\lequa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 7, 16)          │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 3, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 2, 32)          │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 20)             │         1,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,441 (9.54 KB)

...

c:\Users\lequa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\lequa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\lequa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `

MAE: 0.645326966047287
Iteration 9/10 

c:\Users\lequa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\lequa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\lequa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `

MAE: 0.645326966047287
Iteration 10/10 

c:\Users\lequa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\lequa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\lequa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `